## Práctica de scrapping usando la libraria BeautifulSoup

In [1]:
import json

from bs4 import BeautifulSoup
import requests

### Definimos la url que queremos scrapear

In [2]:
URL = "https://jkanime.net/amagami-ss/"

### Realizamos la petición a la página y lo parseamos con BeautifilSoup

In [3]:
response = requests.get(URL)

In [4]:
s = BeautifulSoup(response.text, "lxml")

### Ahora realizamos la busqueda de los links hacía las sugerencias

In [5]:
seccion = s.find_all("div", attrs={"class": "anime__item"})

In [6]:
print(seccion[0].prettify())

<div class="anime__item">
 <a href="https://jkanime.net/photo-kano/" title="Photokano">
  <div class="anime__item__pic homemini set-bg" data-setbg="https://cdn.jkdesu.com/assets/images/animes/image/photo-kano.jpg" style='background-image: url("https://cdn.jkdesu.com/assets/images/animes/image/photo-kano.jpg");'>
   <div class="anime__item__text">
    <ul>
     <li class="peli">
      Anime
     </li>
    </ul>
   </div>
  </div>
 </a>
 <div class="anime__item__text tituloblanco">
  <h5>
   <a href="https://jkanime.net/photo-kano/">
    Photokano
   </a>
  </h5>
 </div>
</div>



In [7]:
sugerencias = {selec.a.get("title"): selec.a.get("href") for selec in seccion}

In [8]:
sugerencias

{'Photokano': 'https://jkanime.net/photo-kano/',
 'Hatsukoi Limited': 'https://jkanime.net/hatsukoi-limited/',
 'Kami nomi zo Shiru Sekai II': 'https://jkanime.net/kami-nomi-zo-shiru-sekai-ii/',
 'Kami nomi zo Shiru Sekai': 'https://jkanime.net/kami-nomi-zo-shiru-sekai/',
 'Clannad': 'https://jkanime.net/clannad/',
 'Yosuga no Sora': 'https://jkanime.net/yosuga-no-sora/'}

### Iremos a cada una de las sugerencias, y nos traeremos el Nombre del Anime, Género y Descripción

In [9]:
animes = []

for _, link in sugerencias.items():
    # Realizamos la petición a cada link
    anime_response = requests.get(link)
    
    # Parseamos la respuesta
    soup = BeautifulSoup(anime_response.text, 'lxml')
    
    # Buscamos la información
    info = soup.find("div", attrs={"class":"anime__details__text"}) 
    
    nombre = info.h3.get_text()
    descripcion = info.p.get_text()
    
    info_genero = info.ul.li.find_next_sibling("li")
    generos = info_genero.find_all("a")
    
    genero = [genero.get_text() for genero in generos]
    
    animes.append(
        {
            "nombre": nombre,
            "descripcion": descripcion,
            "generos": genero
        }
    )


In [10]:
print(animes[0])

{'nombre': 'Photokano', 'descripcion': 'Kazuya, un chico de segundo año, luego de unas vacaciones de verano recibe una cámara digital como regalo, es entonces donde comienza a su fascinación por este gadget desarrollando la fotografía como su nuevo hobby y un empuje a su vida social.', 'generos': ['Colegial', 'Romance', 'Ecchi']}


### Guardamos el resultado de la consulta en un json

In [11]:
with open("animes.json", "w") as file:
    json.dump(animes, file, indent=4, ensure_ascii=False)